In [34]:
import csv
import re
import os
from collections import defaultdict

# 디렉토리 및 출력 파일 경로 설정
log_dir = "./log_data"
csv_output_path = "./h5_summary.csv"

# 로그 파일명 패턴 (trial 포함)
filename_pattern = r"run(?P<trial>\d+)_alpha(?P<alpha>[0-9.]+)_preclude(?P<preclude>\d+)_preserve(?P<preserve>\d+)_temp(?P<temp>\w+)\.log"

# 수집할 RocksDB 통계 항목
target_keys = [
    'rocksdb.flush.write.bytes',
    'rocksdb.compact.write.bytes',
    'rocksdb.bytes.written',
    'rocksdb.compaction.total.time.cpu_micros',
    'rocksdb.bytes.compressed.from',
    'rocksdb.bytes.compressed.to',
    'rocksdb.wal.bytes',
    'rocksdb.stall.micros',
    'rocksdb.memtable.payload.bytes.at.flush',
    'rocksdb.memtable.garbage.bytes.at.flush',
    'rocksdb.last.level.read.bytes',
    'rocksdb.last.level.read.count',
    'rocksdb.non.last.level.read.bytes',
    'rocksdb.non.last.level.read.count',
    'rocksdb.cold.file.read.bytes',
    'rocksdb.cold.file.read.count'

]

latency_fields = [
    'write.P50', 'write.P95', 'write.P99', 'write.P100', 'write.COUNT', 'write.SUM', 'write.AVG',
    'write.micros.P50', 'write.micros.P95', 'write.micros.P99', 'write.micros.P100', 'write.micros.COUNT', 'write.micros.SUM', 'write.micros.AVG',
    'compaction.times.P50', 'compaction.times.P95', 'compaction.times.P99', 'compaction.times.P100', 'compaction.times.COUNT', 'compaction.times.SUM', 'compaction.times.AVG'
]

# SST 레벨 필드 생성
sst_level_fields = []
for i in range(7):
    sst_level_fields.append(f"L{i}_total")
    for temp_type in ["Hot", "Warm", "Cold", "Unknown"]:
        sst_level_fields.append(f"L{i}_{temp_type}")

# 최종 CSV 헤더
header = ['trial', 'alpha', 'preclude', 'preserve', 'temp', 'time(s)', 'keys', 'db_size_bytes'] + \
         sst_level_fields + target_keys + latency_fields

rows = []

# 로그 파일 파싱
for log_file_name in os.listdir(log_dir):
    if not log_file_name.endswith('.log'):
        continue

    match = re.match(filename_pattern, log_file_name)
    if not match:
        print(f"⚠️ Filename {log_file_name} does not match the expected pattern.")
        continue

    trial = int(match.group("trial"))
    alpha = float(match.group("alpha"))
    preclude = int(match.group("preclude"))
    preserve = int(match.group("preserve"))
    temp = match.group("temp")

    log_path = os.path.join(log_dir, log_file_name)

    # 초기값 설정
    stats = {key: 0 for key in target_keys}
    latency = {key: 0 for key in latency_fields}
    sst_files = defaultdict(lambda: {"total": 0, "Hot": 0, "Warm": 0, "Cold": 0, "Unknown": 0})
    time_sec = 0
    keys = 0
    db_size_bytes = 0
    current_level = None
    in_sst_block = False

    with open(log_path, "r") as f:
        for line in f:
            line = line.strip()

            # 시간, 키 수, DB 크기 추출
            if (m := re.match(r"⏱️ Total write time: ([\d\.]+) seconds", line)):
                time_sec = float(m.group(1))
            elif (m := re.match(r"\[💾 Writing (\d+) keys\.\.\.\]", line)):
                keys = int(m.group(1))
            elif (m := re.match(r"DB 디렉토리 사용량: (\d+) bytes", line)):
                db_size_bytes = int(m.group(1))

            # SST 블록 파싱
            elif line.startswith("===== Level별 SST 파일 개수 및 온도별 분포 ====="):
                in_sst_block = True
                continue
            elif line.startswith("==============================================="):
                in_sst_block = False
                current_level = None
                continue
            elif in_sst_block:
                if (m := re.match(r"Level (\d+) ?: ?총 (\d+)개 파일", line)):
                    current_level = int(m.group(1))
                    sst_files[current_level]['total'] = int(m.group(2))
                elif (m := re.match(r"(Hot|Warm|Cold|Unknown) ?: ?(\d+)개", line)):
                    temp_type, count = m.group(1), int(m.group(2))
                    if current_level is not None:
                        sst_files[current_level][temp_type] = count

            # COUNT 항목
            elif (m := re.match(r"(rocksdb\.[\w\.]+)\s+COUNT\s*:\s*(\d+)", line)):
                key, value = m.group(1), int(m.group(2))
                if key in stats:
                    stats[key] = value

            # write latency
            elif (m := re.match(
                r"rocksdb\.db\.write.micros\s+P50\s*:\s*([\d\.]+)\s+P95\s*:\s*([\d\.]+)\s+P99\s*:\s*([\d\.]+)\s+P100\s*:\s*([\d\.]+)\s+COUNT\s*:\s*(\d+)\s+SUM\s*:\s*(\d+)",
                line)):
                p50, p95, p99, p100, count, total = map(float, m.groups())
                latency.update({
                    'write.P50': p50,
                    'write.P95': p95,
                    'write.P99': p99,
                    'write.P100': p100,
                    'write.COUNT': int(count),
                    'write.SUM': int(total),
                    'write.AVG': total / count if count > 0 else 0
                })

            # write micros latency
            elif (m := re.match(
                r"rocksdb\.db\.write\.micros\s+P50\s*:\s*([\d\.]+)\s+P95\s*:\s*([\d\.]+)\s+P99\s*:\s*([\d\.]+)\s+P100\s*:\s*([\d\.]+)\s+COUNT\s*:\s*(\d+)\s+SUM\s*:\s*(\d+)",
                line)):
                p50, p95, p99, p100, count, total = map(float, m.groups())
                latency.update({
                    'write.micros.P50': p50,
                    'write.micros.P95': p95,
                    'write.micros.P99': p99,
                    'write.micros.P100': p100,
                    'write.micros.COUNT': int(count),
                    'write.micros.SUM': int(total),
                    'write.micros.AVG': total / count if count > 0 else 0
                })

            # compaction latency
            elif (m := re.match(
                r"rocksdb\.compaction\.times\.micros\s+P50\s*:\s*([\d\.]+)\s+P95\s*:\s*([\d\.]+)\s+P99\s*:\s*([\d\.]+)\s+P100\s*:\s*([\d\.]+)\s+COUNT\s*:\s*(\d+)\s+SUM\s*:\s*(\d+)",
                line)):
                p50, p95, p99, p100, count, total = map(float, m.groups())
                latency.update({
                    'compaction.times.P50': p50,
                    'compaction.times.P95': p95,
                    'compaction.times.P99': p99,
                    'compaction.times.P100': p100,
                    'compaction.times.COUNT': int(count),
                    'compaction.times.SUM': int(total),
                    'compaction.times.AVG': total / count if count > 0 else 0
                })

    # SST 데이터 한 줄로 변환
    sst_row = []
    for i in range(7):
        level_info = sst_files[i]
        sst_row.append(level_info['total'])
        for temp_type in ['Hot', 'Warm', 'Cold', 'Unknown']:
            sst_row.append(level_info[temp_type])

    # 최종 행 추가
    row = [trial, alpha, preclude, preserve, temp, time_sec, keys, db_size_bytes] + \
          sst_row + \
          [stats[key] for key in target_keys] + \
          [latency[key] for key in latency_fields]

    rows.append(row)

# CSV 저장
with open(csv_output_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    writer.writerows(rows)

print(f"✅ CSV summary saved to {csv_output_path}")


✅ CSV summary saved to ./h5_summary.csv


In [35]:
import pandas as pd
pd.set_option('display.max_columns', None)  # 모든 열 표시
pd.set_option('display.max_rows', None)  # 모든 행 표시

df = pd.read_csv("./h5_summary.csv")
df

,trial,alpha,preclude,preserve,temp,time(s),keys,db_size_bytes,L0_total,L0_Hot,L0_Warm,L0_Cold,L0_Unknown,L1_total,L1_Hot,L1_Warm,L1_Cold,L1_Unknown,L2_total,L2_Hot,L2_Warm,L2_Cold,L2_Unknown,L3_total,L3_Hot,L3_Warm,L3_Cold,L3_Unknown,L4_total,L4_Hot,L4_Warm,L4_Cold,L4_Unknown,L5_total,L5_Hot,L5_Warm,L5_Cold,L5_Unknown,L6_total,L6_Hot,L6_Warm,L6_Cold,L6_Unknown,rocksdb.flush.write.bytes,rocksdb.compact.write.bytes,rocksdb.bytes.written,rocksdb.compaction.total.time.cpu_micros,rocksdb.bytes.compressed.from,rocksdb.bytes.compressed.to,rocksdb.wal.bytes,rocksdb.stall.micros,rocksdb.memtable.payload.bytes.at.flush,rocksdb.memtable.garbage.bytes.at.flush,rocksdb.last.level.read.bytes,rocksdb.last.level.read.count,rocksdb.non.last.level.read.bytes,rocksdb.non.last.level.read.count,rocksdb.cold.file.read.bytes,rocksdb.cold.file.read.count,write.P50,write.P95,write.P99,write.P100,write.COUNT,write.SUM,write.AVG,write.micros.P50,write.micros.P95,write.micros.P99,write.micros.P100,write.micros.COUNT,write.micros.SUM,write.micros.AVG,compaction.times.P50,compaction.times.P95,compaction.times.P99,compaction.times.P100,compaction.times.COUNT,compaction.times.SUM,compaction.times.AVG
0,3,0.9,120,120,cold,80.1453,1000000,490190093,3,0,0,0,3,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,0,3,0,0,0,0,0,619767064,1703799646,16409090136,26965667,46540665295,2311989748,16409090136,65395351,16272153227,3897962679,178011283,213761,1798659055,2187474,0,0,9.981431,19.140115,30.754212,1166823.0,1000000,78007249,78.007249,0,0,0,0,0,0,0,339375.000000,1.488750e+06,2.865000e+06,4648741.0,107,54562000,5.099252e+05
1,3,0.9,15,15,cold,81.2269,1000000,445166284,4,0,0,0,4,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,3,0,620305599,1948591586,16409090136,32185839,51489950820,2556404263,16409090136,66026921,16272153227,3887204988,656939965,797769,1626986966,1979925,302158466,365908,9.825643,20.024498,53.498608,1240045.0,1000000,78985278,78.985278,0,0,0,0,0,0,0,335526.315789,2.500000e+06,5.488000e+06,5834243.0,92,61126819,6.644219e+05
2,2,0.9,120,120,cold,74.9174,1000000,529706241,5,0,0,0,5,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,0,3,0,0,0,0,0,620601117,1671862993,16409090136,26485410,45942118884,2281735990,16409090136,61477760,16272153227,3881301358,190868108,230022,1769127584,2151760,0,0,8.920343,14.893800,28.061768,877218.0,1000000,72874334,72.874334,0,0,0,0,0,0,0,324848.484848,1.526667e+06,2.860000e+06,4524171.0,104,52195696,5.018817e+05
3,1,0.5,0,0,cold,89.6692,1000000,655261253,15,0,0,0,15,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,0,3,6,0,0,6,0,813780014,2386743086,16410542657,36191711,64058397638,3181678696,16410542657,73683571,16339479239,106416754,336932638,405172,2282380671,2773244,336932638,405172,9.752324,20.047564,64.238527,1000505.0,1000000,87247336,87.247336,0,0,0,0,0,0,0,391176.470588,2.800000e+06,9.874213e+06,9874213.0,86,71185652,8.277401e+05
4,1,0.9,15,15,cold,69.0993,1000000,485861920,4,0,0,0,4,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,2,0,0,2,0,619719243,1855441977,16409090136,29588557,49607078393,2463144339,16409090136,54779063,16272153227,3898913852,544013787,660570,1607179019,1954316,170424452,205802,9.443313,19.105810,39.457963,1545619.0,1000000,66941579,66.941579,0,0,0,0,0,0,0,302000.000000,2.400000e+06,4.498708e+06,4498708.0,90,52174322,5.797147e+05
5,2,1.2,120,120,cold,72.1720,1000000,226581221,2,0,0,0,2,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,235451945,746445468,16407152794,12772972,19698573251,977583325,16407152794,59000817,16270230905,11566890722,63135066,76106,833049959,1015725,0,0,9.007107,17.973036,30.336723,934197.0,1000000,70328549,70.328549,0,0,0,0,0,0,0,177741.935484,5.168000e+05,9.688000e+05,1348448.0,168,36428644,2.168372e+05
6,1,0.5,20,20,cold,100.7980,1000000,775259920,7,0,0,0,7,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,2,0,0,0,2,2,0,0,0,2,4,0,0,4,0,810412971,2821357416,16410542657,42698498,72764396696,3613679313,16410542657,84500331,16273594301,107761563,921271217,1115954,2147684411,261

In [31]:
df[['trial', 'alpha', 'preclude', 'rocksdb.last.level.read.bytes', 'rocksdb.non.last.level.read.count']].sort_values(by=['alpha', 'preclude', 'trial'])


,trial,alpha,preclude,rocksdb.last.level.read.bytes,rocksdb.non.last.level.read.count
3,1,0.5,0,336932638,2773244
25,2,0.5,0,327096126,2862907
38,3,0.5,0,317535904,2864988
6,1,0.5,20,921271217,2611275
40,2,0.5,20,1148473214,2619133
26,3,0.5,20,623980619,2756547
20,1,0.5,50,141014680,2516379
18,2,0.5,50,858913360,2647973
29,3,0.5,50,270581589,2862585
14,1,0.5,80,146395327,2607610


In [48]:
import pandas as pd

# 데이터 복사
df = pd.read_csv("./h5_summary.csv")

# === 1. 주요 지표 계산 === #

# WAF (Write Amplification Factor)
df['WAF'] = df.apply(
    lambda row: (
        (row.get('rocksdb.wal.bytes', 0) +
         row.get('rocksdb.flush.write.bytes', 0) +
         row.get('rocksdb.compact.write.bytes', 0))
        / row['rocksdb.bytes.written']
        if row['rocksdb.bytes.written'] != 0 else 0
    ),
    axis=1
)

# Throughput (Bytes Written / Time)
df['throughput'] = df.apply(
    lambda row: (
        row['rocksdb.bytes.written'] / row['time(s)']
        if (row.get('rocksdb.bytes.written', 0) > 0 and row.get('time(s)', 0) > 0)
        else 0
    ),
    axis=1
)

# Stall Time 계산 수정: row.get() 사용
df['stall_flush_total(us)'] = df.apply(
    lambda row: (
        row.get('rocksdb.stall.micros', 0)
    ),
    axis=1
)

# last level read ratio
df['last_level_byte_ratio'] = df.apply(
    lambda row: (
        row.get('rocksdb.last.level.read.bytes', 0) /
        (row.get('rocksdb.last.level.read.bytes', 0) + row.get('rocksdb.non.last.level.read.bytes', 0))
        if (row.get('rocksdb.last.level.read.bytes', 0) + row.get('rocksdb.non.last.level.read.bytes', 0)) > 0
        else 0
    ),
    axis=1
)

df['last_level_count_ratio'] = df.apply(
    lambda row: (
        row.get('rocksdb.last.level.read.count', 0) /
        (row.get('rocksdb.last.level.read.count', 0) + row.get('rocksdb.non.last.level.read.count', 0))
        if (row.get('rocksdb.last.level.read.count', 0) + row.get('rocksdb.non.last.level.read.count', 0)) > 0
        else 0
    ),
    axis=1
)

# alpha, preserve 조합에 따른 case 매핑 함수
def map_case(alpha, preserve):
    if alpha == 0.5:
        mapping = {0: 0, 20: 20, 50: 50, 80: 80, 120: 100}
    elif alpha == 0.9:
        mapping = {0: 0, 15: 20, 40: 50, 65: 80, 120: 100}
    elif alpha == 1.2:
        mapping = {0: 0, 10: 20, 25: 50, 40: 80, 120: 100}
    else:
        return None
    return mapping.get(preserve, None)

# === 2. Compaction / Flush / Stall 관련 분석 === #

analysis_rows = []

for idx, row in df.iterrows():
    total_time = row['time(s)']
    compaction_time = row.get('rocksdb.compaction.total.time.cpu_micros', 0) / 1e6
    compaction_ratio = compaction_time / total_time if total_time > 0 else 0

    stall_micros = row.get('rocksdb.stall.micros', 0)
    stall_flush_total = stall_micros / 1e6

    compressed_from = row.get('rocksdb.bytes.compressed.from', 0)
    compressed_to = row.get('rocksdb.bytes.compressed.to', 0)
    flush_bytes = row.get('rocksdb.flush.write.bytes', 0)
    compact_bytes = row.get('rocksdb.compact.write.bytes', 0)
    latency = row.get('write.AVG', 0)

    # case 계산
    case = map_case(row['alpha'], row['preserve'])

    analysis_rows.append({
        "trial": row["trial"],
        "alpha": row["alpha"],
        "preclude": row["preclude"],
        "preserve": row["preserve"],
        "case": case,
        "time(s)": total_time,
        "keys": row["keys"],
        "db_size_bytes": row["db_size_bytes"],
        
        "L0_total": row["L0_total"],
        "L0_Hot": row["L0_Hot"],
        "L0_Warm": row["L0_Warm"],
        "L0_Cold": row["L0_Cold"],
        "L0_Unknown": row["L0_Unknown"],

        "L1_total": row["L1_total"],
        "L1_Hot": row["L1_Hot"],
        "L1_Warm": row["L1_Warm"],
        "L1_Cold": row["L1_Cold"],
        "L1_Unknown": row["L1_Unknown"],

        "L2_total": row["L2_total"],
        "L2_Hot": row["L2_Hot"],
        "L2_Warm": row["L2_Warm"],
        "L2_Cold": row["L2_Cold"],
        "L2_Unknown": row["L2_Unknown"],

        "L3_total": row["L3_total"],
        "L3_Hot": row["L3_Hot"],
        "L3_Warm": row["L3_Warm"],
        "L3_Cold": row["L3_Cold"],
        "L3_Unknown": row["L3_Unknown"],

        "L4_total": row["L4_total"],
        "L4_Hot": row["L4_Hot"],
        "L4_Warm": row["L4_Warm"],
        "L4_Cold": row["L4_Cold"],
        "L4_Unknown": row["L4_Unknown"],

        "L5_total": row["L5_total"],
        "L5_Hot": row["L5_Hot"],
        "L5_Warm": row["L5_Warm"],
        "L5_Cold": row["L5_Cold"],
        "L5_Unknown": row["L5_Unknown"],

        "L6_total": row["L6_total"],
        "L6_Hot": row["L6_Hot"],
        "L6_Warm": row["L6_Warm"],
        "L6_Cold": row["L6_Cold"],
        "L6_Unknown": row["L6_Unknown"],

        "WAF": row["WAF"],
        "throughput": row["throughput"],
        "last_level_byte_ratio": row["last_level_byte_ratio"],
        "last_level_count_ratio": row["last_level_count_ratio"],
        "compaction_time(s)": compaction_time,
        "compaction_ratio": round(compaction_ratio, 4),
        "stall_flush_total(s)": stall_flush_total,
        "Latency": latency,
        "compressed_from": compressed_from,
        "compressed_to": compressed_to,
        "flush_bytes": flush_bytes,
        "compact_bytes": compact_bytes,
        "write_latency_sum": row.get("write.micros.SUM", 0),
        "compaction_latency_sum": row.get("compaction.times.SUM", 0),

        "cold.read.bytes": row.get('rocksdb.cold.file.read.bytes', 0),
        "cold.read.count": row.get('rocksdb.cold.file.read.count', 0),

        "last.level.read.bytes": row.get('rocksdb.last.level.read.bytes', 0),
        "non.last.level.read.bytes": row.get('rocksdb.non.last.level.read.bytes', 0),
    })

# NaN 값 제거
df.fillna(0, inplace=True)

# 분석 결과 DataFrame 생성
df_analysis = pd.DataFrame(analysis_rows)

# 결과 저장
df_analysis.to_csv('modified_h5_summary.csv', index=False)


In [49]:
df_analysis

,trial,alpha,preclude,preserve,case,time(s),keys,db_size_bytes,L0_total,L0_Hot,L0_Warm,L0_Cold,L0_Unknown,L1_total,L1_Hot,L1_Warm,L1_Cold,L1_Unknown,L2_total,L2_Hot,L2_Warm,L2_Cold,L2_Unknown,L3_total,L3_Hot,L3_Warm,L3_Cold,L3_Unknown,L4_total,L4_Hot,L4_Warm,L4_Cold,L4_Unknown,L5_total,L5_Hot,L5_Warm,L5_Cold,L5_Unknown,L6_total,L6_Hot,L6_Warm,L6_Cold,L6_Unknown,WAF,throughput,last_level_byte_ratio,last_level_count_ratio,compaction_time(s),compaction_ratio,stall_flush_total(s),Latency,compressed_from,compressed_to,flush_bytes,compact_bytes,write_latency_sum,compaction_latency_sum,cold.read.bytes,cold.read.count,last.level.read.bytes,non.last.level.read.bytes
0,3,0.9,120,120,100,80.1453,1000000,490190093,3,0,0,0,3,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,0,3,0,0,0,0,0,1.141602,2.047418e+08,0.090056,0.089021,26.965667,0.3365,65.395351,78.007249,46540665295,2311989748,619767064,1703799646,0,54562000,0,0,178011283,1798659055
1,3,0.9,15,15,20,81.2269,1000000,445166284,4,0,0,0,4,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,3,0,1.156553,2.020155e+08,0.287636,0.287206,32.185839,0.3962,66.026921,78.985278,51489950820,2556404263,620305599,1948591586,0,61126819,302158466,365908,656939965,1626986966
2,2,0.9,120,120,100,74.9174,1000000,529706241,5,0,0,0,5,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,0,3,0,0,0,0,0,1.139707,2.190291e+08,0.097382,0.096576,26.485410,0.3535,61.477760,72.874334,45942118884,2281735990,620601117,1671862993,0,52195696,0,0,190868108,1769127584
3,1,0.5,0,0,0,89.6692,1000000,655261253,15,0,0,0,15,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,0,3,6,0,0,6,0,1.195028,1.830120e+08,0.128634,0.127476,36.191711,0.4036,73.683571,87.247336,64058397638,3181678696,813780014,2386743086,0,71185652,336932638,405172,336932638,2282380671
4,1,0.9,15,15,20,69.0993,1000000,485861920,4,0,0,0,4,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,2,0,0,2,0,1.150841,2.374711e+08,0.252889,0.252619,29.588557,0.4282,54.779063,66.941579,49607078393,2463144339,619719243,1855441977,0,52174322,170424452,205802,544013787,1607179019
5,2,1.2,120,120,100,72.1720,1000000,226581221,2,0,0,0,2,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1.059846,2.273340e+08,0.070449,0.069705,12.772972,0.1770,59.000817,70.328549,19698573251,977583325,235451945,746445468,0,36428644,0,0,63135066,833049959
6,1,0.5,20,20,20,100.7980,1000000,775259920,7,0,0,0,7,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,2,0,0,0,2,2,0,0,0,2,4,0,0,4,0,1.221307,1.628062e+08,0.300190,0.299406,42.698498,0.4236,84.500331,98.286288,72764396696,3613679313,810412971,2821357416,0,80724078,402641220,487095,921271217,2147684411
7,3,0.9,40,40,50,86.3098,1000000,425254101,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2,0,0,0,2,2,0,0,2,0,1.157573,1.901185e+08,0.224733,0.223731,31.601729,0.3661,71.792052,84.075140,51810091141,2573190544,620601240,1965036073,0,62769926,87017088,104344,517266995,1784426591
8,1,1.2,120,120,100,68.0847,1000000,291311751,11,0,0,0,11,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1.060834,2.409815e+08,0.114822,0.113789,13.360403,0.1962,54.855779,66.221754,19995682630,992478553,235456821,762647827,0,35315133,0,0,106324034,819666642
9,2,1.2,25,25,50,64.2916,1000000,217187600,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,1,0,0,1,0,1.072517,2.551990e+08,0.373829,0.373246,15.537420,0.2417,50.893545,62.490198,23862270737,1184071451,234949426,954852127,0,36958643,141943020,172856,416172316,697097710


In [ ]:
df_analysis['Not_L6_Total'] = df_analysis['L0_total'] + df_analysis['L1_total'] + df_analysis['L2_total'] + \
                            df_analysis['L3_total'] + df_analysis['L4_total'] + df_analysis['L5_total']

In [59]:
df_analysis[['case', 'alpha', 'preclude', 'preserve', 'trial', 'Not_L6_Total', 'L6_total', 'last.level.read.bytes', 'non.last.level.read.bytes', 'cold.read.bytes']].groupby(['alpha', 'preclude', 'preserve']).mean().reset_index()

,alpha,preclude,preserve,case,trial,Not_L6_Total,L6_total,last.level.read.bytes,non.last.level.read.bytes,cold.read.bytes
0,0.5,0,0,0.0,2.0,14.333333,6.000000,3.271882e+08,2.331916e+09,3.271882e+08
1,0.5,20,20,20.0,2.0,13.666667,4.000000,8.979084e+08,2.190757e+09,3.866556e+08
2,0.5,50,50,50.0,2.0,19.333333,1.000000,4.235032e+08,2.202845e+09,4.457494e+07
3,0.5,80,80,80.0,2.0,18.666667,0.000000,2.751203e+08,2.228698e+09,0.000000e+00
4,0.5,120,120,100.0,2.0,18.666667,0.000000,3.455350e+08,2.162394e+09,0.000000e+00
5,0.9,0,0,0.0,2.0,9.333333,3.333333,1.929425e+08,1.705286e+09,1.929425e+08
6,0.9,15,15,20.0,2.0,8.666667,2.666667,7.178174e+08,1.580896e+09,3.180140e+08
7,0.9,40,40,50.0,2.0,7.000000,1.333333,5.941549e+08,1.714715e+09,1.524831e+08
8,0.9,65,65,80.0,2.0,11.333333,0.000000,1.763904e+08,1.791749e+09,0.000000e+00
9,0.9,120,120,100.0,2.0,11.666667,0.000000,1.848400e+08,1.778240e+09,0.000000e+00


In [39]:
df_analysis.groupby(['alpha', 'preclude', 'preserve']).min().reset_index()

,alpha,preclude,preserve,trial,case,time(s),keys,db_size_bytes,L0_total,L0_Hot,L0_Warm,L0_Cold,L0_Unknown,L1_total,L1_Hot,L1_Warm,L1_Cold,L1_Unknown,L2_total,L2_Hot,L2_Warm,L2_Cold,L2_Unknown,L3_total,L3_Hot,L3_Warm,L3_Cold,L3_Unknown,L4_total,L4_Hot,L4_Warm,L4_Cold,L4_Unknown,L5_total,L5_Hot,L5_Warm,L5_Cold,L5_Unknown,L6_total,L6_Hot,L6_Warm,L6_Cold,L6_Unknown,WAF,throughput,last_level_byte_ratio,last_level_count_ratio,compaction_time(s),compaction_ratio,stall_flush_total(s),Latency,compressed_from,compressed_to,flush_bytes,compact_bytes,write_latency_sum,compaction_latency_sum,cold.read.bytes,cold.read.count
0,0.5,0,0,1,0,89.6692,1000000,655261253,4,0,0,0,4,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,6,0,0,6,0,1.195028,1.735577e+08,0.118704,0.117441,36.191711,0.3972,73.683571,87.247336,64058397638,3181678696,810458613,2386743086,0,70389097,317535904,381240
1,0.5,20,20,1,20,93.5284,1000000,714172861,4,0,0,0,4,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,2,0,0,0,2,4,0,0,4,0,1.213589,1.533165e+08,0.215606,0.215258,40.579383,0.4236,78.678092,91.071974,70209996041,3487543958,810387378,2694607846,0,74913557,197487861,237131
2,0.5,50,50,1,50,68.2447,1000000,767770112,5,0,0,0,5,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2,0,0,0,2,4,0,0,0,4,0,0,0,0,0,1.175604,1.705200e+08,0.063655,0.062445,31.349235,0.4069,53.878359,65.851186,57684532897,2867293594,810418356,2071279101,0,52632289,0,0
3,0.5,80,80,1,80,66.4617,1000000,735577171,4,0,0,0,4,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,2,0,0,0,2,4,0,0,0,4,0,0,0,0,0,1.179023,1.764592e+08,0.063810,0.062556,31.946739,0.4009,52.209387,64.093152,58828158546,2924083673,810373861,2127363341,0,50993593,0,0
4,0.5,120,120,1,100,66.5402,1000000,718693060,5,0,0,0,5,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2,0,0,0,2,4,0,0,0,4,0,0,0,0,0,1.181956,1.800395e+08,0.114570,0.113433,33.051157,0.4015,51.802798,64.241660,59799960273,2972051525,810482490,2175513432,0,54286268,0,0
5,0.9,0,0,1,0,56.0553,1000000,442607665,4,0,0,0,4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,3,0,0,3,0,1.133066,2.172175e+08,0.097492,0.096745,25.112105,0.3614,41.942264,53.856583,43773355015,2172681355,620309435,1562899626,0,41692923,178685874,215510
6,0.9,15,15,1,20,69.0993,1000000,429154040,4,0,0,0,4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,2,0,1.150841,2.020155e+08,0.252889,0.252619,29.588557,0.3962,54.779063,66.941579,49607078393,2463144339,619719243,1855441977,0,52174322,170424452,205802
7,0.9,40,40,1,50,65.5292,1000000,423155022,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,2,0,0,0,2,0,0,0,0,0,1.136483,1.751263e+08,0.085368,0.084202,26.299266,0.3661,51.226521,63.447288,44861108380,2228346506,620011515,1619547182,0,46835432,0,0
8,0.9,65,65,1,80,66.3597,1000000,459920528,2,0,0,0,2,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,3,0,0,0,3,0,0,0,0,0,1.136134,2.024282e+08,0.083507,0.082530,26.353643,0.3399,51.757488,64.192492,44740719370,2222502232,619774953,1613842371,0,48042256,0,0
9,0.9,120,120,1,100,74.9174,1000000,468635690,3,0,0,0,3,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,2,0,0,0,2,3,0,0,0,3,0,0,0,0,0,1.138720,2.047418e+08,0.090056,0.089021,26.485410,0.3365,61.477760,72.874334,45617118764,2266037673,619767064,1655961468,0,52195696,0,0


In [40]:
df_analysis.groupby(['alpha', 'preclude', 'preserve']).mean().reset_index()

,alpha,preclude,preserve,trial,case,time(s),keys,db_size_bytes,L0_total,L0_Hot,L0_Warm,L0_Cold,L0_Unknown,L1_total,L1_Hot,L1_Warm,L1_Cold,L1_Unknown,L2_total,L2_Hot,L2_Warm,L2_Cold,L2_Unknown,L3_total,L3_Hot,L3_Warm,L3_Cold,L3_Unknown,L4_total,L4_Hot,L4_Warm,L4_Cold,L4_Unknown,L5_total,L5_Hot,L5_Warm,L5_Cold,L5_Unknown,L6_total,L6_Hot,L6_Warm,L6_Cold,L6_Unknown,WAF,throughput,last_level_byte_ratio,last_level_count_ratio,compaction_time(s),compaction_ratio,stall_flush_total(s),Latency,compressed_from,compressed_to,flush_bytes,compact_bytes,write_latency_sum,compaction_latency_sum,cold.read.bytes,cold.read.count
0,0.5,0,0,2.0,0.0,91.982967,1000000.0,7.205374e+08,8.333333,0.0,0.0,0.0,8.333333,1.000000,0.0,0.0,0.0,1.000000,1.000000,0.0,0.0,0.0,1.000000,0.666667,0.0,0.0,0.0,0.666667,1.000000,0.0,0.0,0.0,1.000000,2.333333,0.0,0.0,0.0,2.333333,6.000000,0.0,0.0,6.000000,0.0,1.197309,1.784928e+08,0.123084,0.121868,36.950575,0.401767,75.931667,89.584361,6.483609e+10,3.220349e+09,8.115800e+08,2.426365e+09,0.0,7.124293e+07,3.271882e+08,393116.666667
1,0.5,20,20,2.0,20.0,100.454467,1000000.0,7.694128e+08,5.666667,0.0,0.0,0.0,5.666667,0.666667,0.0,0.0,0.0,0.666667,1.000000,0.0,0.0,0.0,1.000000,1.666667,0.0,0.0,0.0,1.666667,2.333333,0.0,0.0,0.0,2.333333,2.333333,0.0,0.0,0.0,2.333333,4.000000,0.0,0.0,4.000000,0.0,1.223272,1.638611e+08,0.287835,0.287279,43.217944,0.430267,84.641960,97.940982,7.339850e+10,3.645483e+09,8.104362e+08,2.853583e+09,0.0,8.008794e+07,3.866556e+08,467690.666667
2,0.5,50,50,2.0,50.0,84.441200,1000000.0,8.553033e+08,9.000000,0.0,0.0,0.0,9.000000,0.666667,0.0,0.0,0.0,0.666667,1.333333,0.0,0.0,0.0,1.333333,1.666667,0.0,0.0,0.0,1.666667,2.666667,0.0,0.0,0.0,2.666667,4.000000,0.0,0.0,0.0,4.000000,1.000000,0.0,0.0,1.000000,0.0,1.196149,1.985683e+08,0.149824,0.149069,36.526046,0.434500,69.985011,82.082825,6.444537e+10,3.202325e+09,8.115453e+08,2.407374e+09,0.0,6.394139e+07,4.457494e+07,53536.666667
3,0.5,80,80,2.0,80.0,82.082800,1000000.0,7.812257e+08,8.000000,0.0,0.0,0.0,8.000000,1.000000,0.0,0.0,0.0,1.000000,0.666667,0.0,0.0,0.0,0.666667,1.333333,0.0,0.0,0.0,1.333333,2.333333,0.0,0.0,0.0,2.333333,5.333333,0.0,0.0,0.0,5.333333,0.000000,0.0,0.0,0.000000,0.0,1.188898,2.041550e+08,0.107778,0.106272,35.008781,0.431333,67.367461,79.808189,6.207402e+10,3.085172e+09,8.104635e+08,2.289451e+09,0.0,6.384037e+07,0.000000e+00,0.000000
4,0.5,120,120,2.0,100.0,77.477200,1000000.0,7.664324e+08,8.333333,0.0,0.0,0.0,8.333333,0.666667,0.0,0.0,0.0,0.666667,0.666667,0.0,0.0,0.0,0.666667,1.333333,0.0,0.0,0.0,1.333333,2.333333,0.0,0.0,0.0,2.333333,5.333333,0.0,0.0,0.0,5.333333,0.000000,0.0,0.0,0.000000,0.0,1.188898,2.154096e+08,0.137984,0.136510,34.978467,0.456800,62.566196,75.119659,6.207732e+10,3.085133e+09,8.104997e+08,2.289426e+09,0.0,6.057880e+07,0.000000e+00,0.000000
5,0.9,0,0,2.0,0.0,63.717367,1000000.0,4.871440e+08,5.000000,0.0,0.0,0.0,5.000000,0.000000,0.0,0.0,0.0,0.000000,0.666667,0.0,0.0,0.0,0.666667,1.000000,0.0,0.0,0.0,1.000000,1.000000,0.0,0.0,0.0,1.000000,1.666667,0.0,0.0,0.0,1.666667,3.333333,0.0,0.0,3.333333,0.0,1.135268,2.618260e+08,0.101574,0.100597,26.096518,0.414900,49.384187,61.553092,4.448801e+10,2.208498e+09,6.204994e+08,1.599117e+09,0.0,4.613192e+07,1.929425e+08,232253.000000
6,0.9,15,15,2.0,20.0,75.797800,1000000.0,4.533941e+08,4.000000,0.0,0.0,0.0,4.000000,0.333333,0.0,0.0,0.0,0.333333,0.666667,0.0,0.0,0.0,0.666667,1.000000,0.0,0.0,0.0,1.000000,1.000000,0.0,0.0,0.0,1.000000,1.666667,0.0,0.0,0.0,1.666667,2.666667,0.0,0.0,2.666667,0.0,1.158156,2.174686e+08,0.309186,0.308776,31.694799,0.418867,60.946513,73.598330,5.200995e+10,2.582142e+09,6.202087e+08,1.974983e+09,0.0,5.879059e+07,3.180140e+08,385492.333333
7,0.9,40,40,2.0,50.0,81.845867,1000000.0,4.426622e+08,2.333333,0.0,0.0,0.0,2.333333,0.000000,0.0,0.0,0.0,0.000000,0.333333,0.0,0.0,0.0,0.333333,0.666667,0.0,0.0,0.0,0.666667,1.333333,0.0,0.0,0.0,1.333333,2.333333,0.0,0.0,0.0,2.333333,1.333333,0.0,0.0,1.333333,0.0,1.158735,2.052179e+08,0.238809,0.237889,31.664834,0.387767

In [45]:
df_analysis['sst_byte'] = df_analysis['cold.read.bytes'] / df_analysis['cold.read.count']
df_analysis[['alpha', 'preclude', 'preserve', 'trial', 'cold.read.bytes', 'cold.read.count', 'sst_byte']].sort_values(by=['alpha', 'preclude', 'trial'])

,alpha,preclude,preserve,trial,cold.read.bytes,cold.read.count,sst_byte
3,0.5,0,0,1,336932638,405172,831.579275
25,0.5,0,0,2,327096126,392938,832.436990
38,0.5,0,0,3,317535904,381240,832.902906
6,0.5,20,20,1,402641220,487095,826.617436
40,0.5,20,20,2,559837625,678846,824.690173
26,0.5,20,20,3,197487861,237131,832.821778
20,0.5,50,50,1,0,0,NaN
18,0.5,50,50,2,133724822,160610,832.605828
29,0.5,50,50,3,0,0,NaN
14,0.5,80,80,1,0,0,NaN
